In [6]:
#from meteo_scraping import data_downloader, compute_coordinates
from data_manage import data_handler, remove_header, replace_failed_download
import numpy as np
import os
import json

Define all necessary parameters to download and handle data

In [8]:
#related to data download 
start_date, end_date = "2017-01-01","2022-01-01"
reference_coord = (53.042004, 7.105163)
d_x, d_y  = 17, 17
km_on_x, km_on_y = 30*(d_x-1), 30*(d_y-1)


#related to data handle
window_enc = 24
window_dec = 8
horizon = 1
jump = 9
keep_columns_inputs = ['windspeed_10m (km/h)',
                       'windspeed_100m (km/h)',
                       'windgusts_10m (km/h)',
                       'winddirection_10m (°)',
                       'winddirection_100m (°)', #wind related
                       
                       'temperature_2m (°C)',
                       'apparent_temperature (°C)', #temperature related
                       
                       #'relativehumidity_2m (%)', #humidity related
                       
                       'diffuse_radiation (W/m²)', 
                       'shortwave_radiation (W/m²)',
                       'direct_radiation (W/m²)', #radiation related
                       
                       'pressure_msl (hPa)',
                       'surface_pressure (hPa)', #pressure related
                       
                       'cloudcover_low (%)',
                       'cloudcover_mid (%)', 
                       'cloudcover_high (%)'] #cloudover related

keep_columns_labels = ['windspeed_100m (km/h)']

In [3]:
assert False, 'take a moment to look if numpy data (or csv files) and json meta-info are available'

AssertionError: take a moment to look if numpy data (or csv files) and json meta-info are available

In [4]:
is_csv_data_available = True
is_npy_data_available = False
is_json_metainfo_available = False

data_path = "C:\Stuffs\Tesi Samuele\ConvLSTM\data" #specify without ending \  path for csv files
npy_path = 'C:\Stuffs\Tesi Samuele\ConvLSTM\data_npy' #path for npy data
json_path = 'C:\Stuffs\Tesi Samuele\ConvLSTM\info_json' #path for json meta info

In [5]:
#coordinates = compute_coordinates( reference_coord, km_on_x, km_on_y, d_x, d_y ) 

my_file = open("coordinates_17x17.txt", "r")
coordinates_str= my_file.read()
my_file.close()
coordinates_str=coordinates_str[1:-1].replace('(','').replace(')','').replace(' ','').replace('\n','').split(',')
coordinates=[]
for i in range(int(len(coordinates_str)/2)):
    coordinates.append( ( float(coordinates_str[2*i]), float(coordinates_str[2*i + 1]) ) )


Eventually download data (if not yet downloaded); data downloader always returns a list with coordinates tuples, used in data handle 

In [6]:
try:
    assert not is_csv_data_available
    downloaded_coordinates, failed_download = data_downloader(start_date, end_date, coordinates, data_path)
    remove_header(data_path)
    try:
        assert len(failed_download) == 0
    except:
        print('some files have not been downloaded properly!')
        replace_failed_download ((d_x, d_y), coordinates, failed_download )
        assert len(os.listdir(data_path)) == d_x*d_y, 'still something wrong'
        is_csv_data_available = True
        pass
except:
    pass

Create tensors as numpy arrays from downloaded csv files, if not available

In [9]:
if not is_npy_data_available:
    train_data, test_data, info = data_handler (  data_path, coordinates, d_x, d_y, 
                        window_enc, horizon, window_dec, jump,
                        keep_columns_inputs,
                        keep_columns_labels  )

If numpy data are not already available, store them:

In [8]:
try:
    assert not is_npy_data_available
    enc_in_train, dec_in_train, dec_out_train = train_data
    np.save(f"{npy_path}\enc_in_train.npy", enc_in_train)
    np.save(f"{npy_path}\dec_in_train.npy", dec_in_train)
    np.save(f"{npy_path}\dec_out_train.npy", dec_out_train)


    enc_in_test, dec_in_test, dec_out_test = test_data
    np.save(f"{npy_path}\enc_in_test.npy", enc_in_test)
    np.save(f"{npy_path}\dec_in_test.npy", dec_in_test)
    np.save(f"{npy_path}\dec_out_test.npy", dec_out_test)


    enc_in_min, enc_in_max, dec_in_min, dec_in_max = [ info[name] for name in list(info.keys())[2:] ]
    np.save(f"{npy_path}\enc_in_min.npy", enc_in_min)
    np.save(f"{npy_path}\enc_in_max.npy", enc_in_max)
    np.save(f"{npy_path}\dec_in_min.npy", dec_in_min)
    np.save(f"{npy_path}\dec_in_max.npy", dec_in_max)
    
    n_inputs, n_outputs = [info[name] for name in list(info.keys())[:2] ]
    np.save(f"{npy_path}\in_out_features.npy", np.array([n_inputs, n_outputs]))
    
    is_npy_data_available  = True 
    
except:
    pass


In [21]:
try:
    assert is_npy_data_available
    n_inputs, n_outputs = np.load(npy_path+'\in_out_features.npy')
except:
    pass

In [33]:
try: 
    assert not is_json_metainfo_available
    assert len(os.listdir(json_path)) == 0, 'you told json meta-info are not available, but something was found in json meta-info path'
    meta_info = {}
    meta_info['start_date']=start_date
    meta_info['end_date']=end_date
    meta_info['reference_coord']=reference_coord
    meta_info['window_enc']=window_enc
    meta_info['window_dec']=window_dec
    meta_info['horizon']=horizon
    meta_info['jump']=jump
    meta_info['keep_columns_inputs']=keep_columns_inputs
    meta_info['keep_columns_labels']=keep_columns_labels
    meta_info['d_x'] = d_x
    meta_info['d_y'] = d_y
    meta_info['km_on_x']=km_on_x
    meta_info['km_on_y']=km_on_y
    meta_info['n_inputs']=n_inputs.item()
    meta_info['n_outputs']=n_outputs.item()
    meta_info['coordinates']=coordinates
except:
    pass

In [34]:
with open(json_path+'\data_meta_info.json', 'w') as fp:
    json.dump(meta_info, fp, indent=4)

## Instructions to run on Colab

1) load scripts on Drive ('/content/drive/MyDrive/TESI_SAMU/MyTests/Scripts');
2) load data_meta_info.json on Drive ('/content/drive/MyDrive/TESI_SAMU/MyTests/MetaInfo');
3) load npy data on Drive ('/content/drive/MyDrive/TESI_SAMU/MyTests/Data');
4) run ConvLSTM_train.ipynb on Colab to train the model (with GPU device);
5) run ConvLSTM_forecast.ipynb on Colab to get test(forecast) results (with CPU device);